# Protein sequence library builder for AVCR and partners

## List of target proteins

* ALK1
* ALK2
* ALK3
* ALK4
* ALK5
* ALK6
* ALK7
* BMPR2
* ActR-IIA
* ActR-IIB
* TGFR2
* AMH-RII
* EGFR (control) 

In [1]:
#Enviroment init
from Bio.Blast import NCBIWWW, NCBIXML
from Bio import SeqIO, AlignIO, ExPASy
from Bio.Align.Applications import MafftCommandline as mafft
from bokeh.models import ColumnDataSource, CDSView, BooleanFilter
from bokeh.models import Title, LinearAxis, Range1d
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
import numpy as np
import time, os, glob
output_notebook()

Loading BokehJS ...

In [2]:
target_handle ={
    "ALK1":"P37023",
    "ALK2":"Q04771",
    "ALK3":"P36894",
    "ALK4":"P36896",
    "ALK5":"P36897",
    "ALK6":"Q05438",
    "ALK7":"Q8NER5",
    "BMPR2":"Q13873",
    "ActR-IIA":"Q7SXW6",
    "ActR-IIB":"Q56A35",
    "TGFR2":"P37173",
    "AMH-RII":"Q16671",
    "EGFR":"P00533"
}

In [21]:
try:
    os.mkdir("Targets")
except:
    pass
for target in target_handle.keys():
    print("Now getting",target,"sequence file")
    filename = "Targets/"+ target + ".fasta"
    handle = ExPASy.get_sprot_raw(target_handle[target])
    temp = SeqIO.read(handle, "swiss")
    print("Writing",target,"fasta file")
    SeqIO.write(temp, filename, "fasta")
    print("Done!")
    time.sleep(0.5) #Some rest to not make it look like a DDoS

Now getting ALK1 sequence file
Writing ALK1 fasta file
Done!
Now getting ALK2 sequence file
Writing ALK2 fasta file
Done!
Now getting ALK3 sequence file
Writing ALK3 fasta file
Done!
Now getting ALK4 sequence file
Writing ALK4 fasta file
Done!
Now getting ALK5 sequence file
Writing ALK5 fasta file
Done!
Now getting ALK6 sequence file
Writing ALK6 fasta file
Done!
Now getting ALK7 sequence file
Writing ALK7 fasta file
Done!
Now getting BMPR2 sequence file
Writing BMPR2 fasta file
Done!
Now getting ActR-IIA sequence file
Writing ActR-IIA fasta file
Done!
Now getting ActR-IIB sequence file
Writing ActR-IIB fasta file
Done!
Now getting TGFR2 sequence file
Writing TGFR2 fasta file
Done!
Now getting AMH-RII sequence file
Writing AMH-RII fasta file
Done!
Now getting EGFR sequence file
Writing EGFR fasta file
Done!


In [2]:
os.getcwd()

'/home/erquiroga/Documentos/AVCR/Sequences'

In [12]:
target_name = target_handle.keys()
rewrite = True #Keep False if not needed
tol = 1e2
hitlist = 2000 #Default is 50
for target in target_name:
    filename ="Targets/"+target+".fasta"
    result_filename = "Blast_XML/" + target + "_blast.xml"
    if os.path.exists(result_filename):
        if rewrite:
            print("Running BLAST against SwissProt using %s as query" %(target))
            with open(filename) as file:
                query =file.read()
                handle = NCBIWWW.qblast("blastp","swissprot",query,
                                        expect=tol, hitlist_size=hitlist)
                print("Rewriting BLAST results of %s" %(target))
                open(result_filename,"w+").write(handle.read())
                print("Done!") 
        else:
            print("Results for %s were found, skiping..." %(target))
    else:
        print("Running BLAST against SwissProt using %s as query" %(target))
        with open(filename) as file:
            query =file.read()
            handle = NCBIWWW.qblast("blastp","swissprot",query,
                                    expect=tol, )
            print("Saving BLAST results of %s" %(target))
            open(result_filename,"w+").write(handle.read())
            print("Done!")
    time.sleep(0.5)

Running BLAST against SwissProt using ALK1 as query
Rewriting BLAST results of ALK1
Done!
Running BLAST against SwissProt using ALK2 as query
Rewriting BLAST results of ALK2
Done!
Running BLAST against SwissProt using ALK3 as query
Rewriting BLAST results of ALK3
Done!
Running BLAST against SwissProt using ALK4 as query
Rewriting BLAST results of ALK4
Done!
Running BLAST against SwissProt using ALK5 as query
Rewriting BLAST results of ALK5
Done!
Running BLAST against SwissProt using ALK6 as query
Rewriting BLAST results of ALK6
Done!
Running BLAST against SwissProt using ALK7 as query
Rewriting BLAST results of ALK7
Done!
Running BLAST against SwissProt using BMPR2 as query
Rewriting BLAST results of BMPR2
Done!
Running BLAST against SwissProt using ActR-IIA as query
Rewriting BLAST results of ActR-IIA
Done!
Running BLAST against SwissProt using ActR-IIB as query
Rewriting BLAST results of ActR-IIB
Done!
Running BLAST against SwissProt using TGFR2 as query
Rewriting BLAST results of TG

In [10]:
mylist = [f for f in glob.glob("Blast_XML/*.xml")]

In [25]:
def BLASTGraphicEval(blastfile,cutoff=False,cutoff_val=50):
    from Bio.Blast import NCBIXML
    from bokeh.models import ColumnDataSource, CDSView, BooleanFilter
    from bokeh.models import Title, LinearAxis, Range1d
    from bokeh.plotting import figure, show
    from bokeh.io import export_png
    import numpy as np
    
    #Extracting Evalues
    result_handle = open(blastfile)
    record = NCBIXML.read(result_handle)

    evalue=[]
    for aln in record.alignments:
        hsp = aln.hsps[0] #just first hsp to avoid multiple hit scanning
        val = hsp.expect
        evalue.append(hsp.expect)
    
    #Calulating log of Evalue
    log_eval=[]
    for i in evalue:
        if i == 0:
            log_eval.append(-420) #Arbitrary value, because log(0) is undefined
        else:
            log_eval.append(np.log(i))
    #Calculating Difference of Evalues (1st derivate)
    d_eval= list(np.diff(log_eval))
    d_eval.insert(0,d_eval[0]) #Adding first value as 0 to square up with the index
    index = list(range(len(evalue)))
    #print("Current lenght is",len(evalue))
    #Saving data in bokeh plot data
    data = {
        "index": index,
        "evalue": log_eval,
        "derivate": d_eval
    }
    source = ColumnDataSource(data=data)
    
    #Generating plot
    minY=min(data["derivate"])
    maxY=max(data["derivate"])
    name=blastfile.split("/")[1].split("_")[0]
    if cutoff:
        #Generating plot figure
        p = figure(plot_height=600, plot_width=800,
                   title="{0} Blast against SwissProt with cutoff at {1}".format(name,cutoff_val), 
                   title_location="above", x_range=Range1d(start=0, end=cutoff_val),
                   toolbar_location=None)
        #Adding titles and labels
        p.title.align = "center"
        p.xaxis.axis_label = "Sequence Index"
        p.yaxis.axis_label = "E-Value (log)"
        #Filtering
        booleans = [True if index < cutoff_val else False for index in source.data['index']]
        view = CDSView(source=source, filters=[BooleanFilter(booleans)])
        #Plotting lines
        p.extra_y_ranges = {"linear": Range1d(start=minY-10, end=maxY+10)}
        p.line(x="index", y="derivate", source=source, line_width=1, line_color="red", view=view, y_range_name="linear", line_dash='dashed',legend="1st Derivate" )
        p.line(x="index", y="evalue", source=source, view=view, line_width=2, line_color="blue",legend="E-Value")
        #Export to file
        export_png(p, filename="Plots/plot_{}_cutoff.png".format(name))
    else:
        #Generating plot figure
        p = figure(plot_height=600, plot_width=800,
                   title="%s Blast against SwissProt"%(name), title_location="above",
                   toolbar_location=None)
        #Adding titles and labels
        p.title.align = "center"
        p.xaxis.axis_label = "Sequence Index"
        p.yaxis.axis_label = "E-Value (log)"
        #Plotting lines
        p.extra_y_ranges = {"linear": Range1d(start=minY-10, end=maxY+10)}
        p.line(x="index", y="derivate", source=source, line_width=2, line_color="red", y_range_name="linear", line_dash='dashed',legend="1st Derivate")
        p.line(x="index", y="evalue", source=source, line_width=2, line_color="blue",legend="E-Value")
        #Export to file
        export_png(p, filename="Plots/plot_{}.png".format(name))
    

In [23]:
mylist

['Blast_XML/ALK7_blast.xml',
 'Blast_XML/ALK6_blast.xml',
 'Blast_XML/TGFR2_blast.xml',
 'Blast_XML/ALK5_blast.xml',
 'Blast_XML/ALK4_blast.xml',
 'Blast_XML/ActR-IIA_blast.xml',
 'Blast_XML/EGFR_blast.xml',
 'Blast_XML/ALK2_blast.xml',
 'Blast_XML/AMH-RII_blast.xml',
 'Blast_XML/ActR-IIB_blast.xml',
 'Blast_XML/BMPR2_blast.xml',
 'Blast_XML/ALK3_blast.xml',
 'Blast_XML/ALK1_blast.xml']

In [27]:
for file in mylist:
    print("Plotting "+file)
    BLASTGraphicEval(file)

Plotting Blast_XML/ALK7_blast.xml
Plotting Blast_XML/ALK6_blast.xml
Plotting Blast_XML/TGFR2_blast.xml
Plotting Blast_XML/ALK5_blast.xml
Plotting Blast_XML/ALK4_blast.xml
Plotting Blast_XML/ActR-IIA_blast.xml
Plotting Blast_XML/EGFR_blast.xml
Plotting Blast_XML/ALK2_blast.xml
Plotting Blast_XML/AMH-RII_blast.xml
Plotting Blast_XML/ActR-IIB_blast.xml
Plotting Blast_XML/BMPR2_blast.xml
Plotting Blast_XML/ALK3_blast.xml
Plotting Blast_XML/ALK1_blast.xml


In [28]:
def dataExtraction(blastfile):
    result_handle = open(blastfile)
    record = NCBIXML.read(result_handle)

    evalue=[]
    for aln in record.alignments:
        hsp = aln.hsps[0]
        val = hsp.expect
        evalue.append(hsp.expect)
    #Calulating log of Evalue
    log_eval=[]
    for i in evalue:
        if i == 0:
            log_eval.append(-420)
        else:
            log_eval.append(np.log(i))
    #Calculating Difference of Evalues (1st derivate)
    d_eval= list(np.diff(log_eval))
    d_eval.insert(0,d_eval[0])
    index = list(range(len(evalue)))
    return (log_eval,d_eval,index)

In [29]:
datalist = []
for file in mylist:
    temp = dataExtraction(file)
    datalist.append(temp)

In [30]:
cutoff_list = []
for data in datalist:
    d_max = max(data[1])
    index = data[1].index(d_max)
    print("maximun change is at",index,"with value of",d_max)
    d_data = data[1][index+1:]
    length=len(d_data)-1
    for i in range(length):
        if (d_data[i-1] < d_data[i] and d_data[i+1] < d_data[i]):
            index_d = data[1].index(d_data[i])
            if index_d - index < 100:
                continue
            else:
                break
    cutoff_list.append(index_d)
    print("break at",index_d)

maximun change is at 26 with value of 144.80900769144122
break at 126
maximun change is at 26 with value of 110.48441612138606
break at 128
maximun change is at 5 with value of 210.50922308261212
break at 106
maximun change is at 26 with value of 153.4957662684573
break at 127
maximun change is at 26 with value of 146.04223811055238
break at 126
maximun change is at 19 with value of 125.241005903151
break at 121
maximun change is at 22 with value of 132.46182932044542
break at 127
maximun change is at 26 with value of 161.99756249093704
break at 127
maximun change is at 4 with value of 221.22218483755333
break at 105
maximun change is at 19 with value of 125.46788518229431
break at 119
maximun change is at 2 with value of 212.2317330984363
break at 105
maximun change is at 26 with value of 129.4240200416611
break at 126
maximun change is at 26 with value of 136.17347551993035
break at 126


In [31]:
for i in range(len(cutoff_list)):
    print("Plotting",mylist[i])
    BLASTGraphicEval(mylist[i],cutoff=True,cutoff_val=cutoff_list[i])

Plotting Blast_XML/ALK7_blast.xml
Plotting Blast_XML/ALK6_blast.xml
Plotting Blast_XML/TGFR2_blast.xml
Plotting Blast_XML/ALK5_blast.xml
Plotting Blast_XML/ALK4_blast.xml
Plotting Blast_XML/ActR-IIA_blast.xml
Plotting Blast_XML/EGFR_blast.xml
Plotting Blast_XML/ALK2_blast.xml
Plotting Blast_XML/AMH-RII_blast.xml
Plotting Blast_XML/ActR-IIB_blast.xml
Plotting Blast_XML/BMPR2_blast.xml
Plotting Blast_XML/ALK3_blast.xml
Plotting Blast_XML/ALK1_blast.xml


In [44]:
os.mkdir("Sequence_Library")

In [10]:
def FastaBuilder(blastfile,cutoff=50):
    import sys
    import time
    from Bio.Align import MultipleSeqAlignment


    def progressbar(it, prefix="", size=60, file=sys.stdout):
        count = len(it)
        def show(j):
            x = int(size*j/count)
            file.write("%s[%s%s] %i/%i\r" % (prefix, "#"*x, "."*(size-x), j, count))
            file.flush()        
        show(0)
        for i, item in enumerate(it):
            yield item
            show(i+1)
        file.write("\n")
        file.flush()

    i = 0
    ids = []
    name=blastfile.split("/")[1].split("_")[0]
    filename ="Sequence_Library/{0}.fa".format(name)
    if os.path.exists(filename):
        print("Saved File found, skipping...")
    else:    
        result_handle = open(blastfile)
        record = NCBIXML.read(result_handle)
        for alignment in record.alignments:
            for hsp in alignment.hsps:
                temp_str = alignment.title
                tag = temp_str.split("|")[1].split(".")[0]
                ids.append(tag)
                if i > cutoff:
                    break
                i +=1
        temp = []
        for i in progressbar(range(len(ids[:cutoff])), "Downloading:", 100):
            try:
                handle = ExPASy.get_sprot_raw(ids[i])
                seq = SeqIO.read(handle, "swiss")
                temp.append(seq)
            except ValueError:
                print("Culprit is",tag, "on position", ids.index(tag))
        print("Saving file...")
        SeqIO.write(temp, "Sequence_Library/{0}.fa".format(name), "fasta")



In [11]:
for i in range(len(cutoff_list)):
    print("Working with",mylist[i])
    FastaBuilder(mylist[i],cutoff=cutoff_list[i])

Working with Blast_XML/AMH-RII_blast.xml
Saved File found, skipping...
Working with Blast_XML/ALK5_blast.xml
Saved File found, skipping...
Working with Blast_XML/ALK3_blast.xml
Saved File found, skipping...
Working with Blast_XML/BMPR2_blast.xml
Saved File found, skipping...
Working with Blast_XML/ALK4_blast.xml
Saved File found, skipping...
Working with Blast_XML/ALK2_blast.xml
Saved File found, skipping...
Working with Blast_XML/TGFR2_blast.xml
Saved File found, skipping...
Working with Blast_XML/ActR-IIA_blast.xml
Saved File found, skipping...
Working with Blast_XML/ALK6_blast.xml
Saved File found, skipping...
Working with Blast_XML/ALK7_blast.xml
Saved File found, skipping...
Working with Blast_XML/ActR-IIB_blast.xml
Saved File found, skipping...
Working with Blast_XML/EGFR_blast.xml
Saved File found, skipping...
Working with Blast_XML/ALK1_blast.xml
Saved File found, skipping...


In [3]:
seq_list = [f for f in glob.glob("Sequence_Library/*.fa")]

In [6]:
os.mkdir("MSA")

In [8]:
for file in seq_list:
    mafft_cmd = mafft(input=file)
    name=file.split("/")[1].split(".")[0]
    stdout, stderr = mafft_cmd()
    with open("MSA/{0}_aligned.fa".format(name), "w+") as handle:
        handle.write(stdout)